# Chatbot Using Retrieval Augmented Generation (RAG)

I want an assistant that can tell me if the scientific information that I am receiving is current. ChatGPT3.5 is only trained to January 2022. So, anything past that, it will not have the information or ... it will lie:)

## Import Libraries

In [1]:
from scipy.spatial.distance import cosine
import numpy as np
from openai import OpenAI # for calling the OpenAI API
import pandas as pd
import pprint
import re
import requests
import sys

## Data Wrangling

We will use Wikipedia as our source. Put the RAG text into a Pandas dataframe.

In [2]:
# I was originally going to do multiple topics, but decided to just do one
topics = ['2024 in science']
topics

['2024 in science']

In [3]:
def get_wikipedia_content(topic):
    # Base URL of the Wikimedia API
    base_url = "https://en.wikipedia.org/w/api.php"
    
    # Parameters for the API request
    params = {
        "action": "query",
        "format": "json",
        "prop": "extracts",
        "titles": topic
    }
    
    # Send a GET request to the Wikimedia API
    response = requests.get(base_url, params=params)\
    
    # Parse the JSON response
    data = response.json()
    
    return data

In [4]:
# Example usage
topic = topics[0]
data = get_wikipedia_content(topic)
pprint.pprint(data)

{'batchcomplete': '',
 'query': {'pages': {'74041015': {'extract': '<p class="mw-empty-elt">\n'
                                             '</p><p>The following scientific '
                                             'events occurred or are scheduled '
                                             'to occur in 2024.\n'
                                             '</p>\n'
                                             '<meta '
                                             'property="mw:PageProp/toc">\n'
                                             '<h2><span '
                                             'id="Events">Events</span></h2>\n'
                                             '<h3><span '
                                             'id="January">January</span></h3>\n'
                                             '<ul><li>2 January – The Japan '
                                             'Meteorological Agency (JMA) '
                                             'publishes its

In [5]:
# Extracting the key from the nested JSON object. We just want the first hit.
# We will need this later.
pages = data['query']['pages']
page_key = list(pages.keys())[0]
page_key

'74041015'

In [6]:
def extract_events(html):
    events = {}
    current_date = None

    # Find all occurrences of dates in the format "digit space Month"
    date_pattern = r'\b(\d{1,2}\s(?:January|February|March|April|May|June|July|August|September|October|November|December))\b'
    dates = re.findall(date_pattern, html)
    
    # Define the pattern to match HTML tags
    html_tags_pattern = re.compile(r'<.*?>')

    for date in dates:
        if current_date is not None:
            current_event = []

        current_date = date

        # Find the text between two consecutive dates
        event_pattern = re.escape(date) + r'(.*?)' + re.escape(dates[dates.index(date) + 1] if dates.index(date) < len(dates) - 1 else '')
        event_text_match = re.search(event_pattern, html, re.DOTALL)
        
        # Where there are multiple events on a date split those into a list.
        if event_text_match:
            event_text = event_text_match.group(1)
            if event_text.startswith('\n<ul><li>'):
                event_text_list = event_text.split('</li>')
                cleaned_value = [remove_html_and_newlines(item) for item in event_text_list]
                cleaned_value = [value for value in cleaned_value if value != '']  
                event_text = cleaned_value
            else:
                event_text = [remove_html_and_newlines(event_text)]

            events[current_date] = event_text

    return events

In [7]:
def remove_html_and_newlines(text):
    
    # Remove HTML tags
    cleaned_text = re.sub(r'<.*?>', '', text)
    
    # Remove newline characters
    cleaned_text = cleaned_text.replace('\n', '')
    
    # Remove any extra whitespace
    cleaned_text = ' '.join(cleaned_text.split())
    
    return cleaned_text

In [8]:
# Put into events_dict
data = get_wikipedia_content(topics[0])

# We just want the extract
extract = data['query']['pages'][page_key]['extract']

# Create the dictionary
events_dict = extract_events(extract)

In [9]:
# Check and see what they look like.
events_dict['3 January']

['– The first functional semiconductor made from graphene is created.']

In [10]:
# See what happens when there is more than one event on a date
print(len(events_dict['9 January']))
events_dict['9 January']

5


['Scientists report studies which seem to support the hypothesis that life may have begun in a shallow lake rather than otherwise - perhaps somewhat like a "warm little pond" originally proposed by Charles Darwin.',
 'A group of scientists from around the globe have charted paradigm shifting restorative pathways to mitigate the worst effects of climate change and biodiversity loss with a strong emphasis on environmental sustainability, human wellbeing and reducing social and economic inequality.',
 'Researchers have discovered a new phase of matter, named a "light-matter hybrid", which may reshape understanding of how light interacts with matter.',
 "A study of proteins in cerebrospinal fluid indicates there are five subtypes of Alzheimer's disease, suggesting it to be likely that subtype-specific treatments are required.",
 'A study finds seaweed farming could be set up as a resilient food solution within roughly a year in abrupt sunlight reduction scenarios such as after a nuclear wa

In [11]:
# Extracting all values from events_dict with keys prepended
all_values = [f"{key} {value}" for key, values_list in events_dict.items() for value in values_list]
length = len(all_values)

In [12]:
# Create the dataframe to hold the text
df = pd.DataFrame(data={'text': all_values}, index=range(length))
df

,text
0,2 January – The Japan Meteorological Agency (J...
1,3 January – The first functional semiconductor...
2,4 January – A review indicates digital rectal ...
3,5 January Scientists report that newborn galax...
4,5 January An analysis of sugar-sweetened bever...
...,...
99,8 May Atmospheric gases surrounding 55 Cancri ...
100,9 May – A record annual increase in atmospheri...
101,10 May – A series of solar storms and intense ...
102,"13 May – OpenAI reveals GPT-4o, its latest AI ..."


## Custom Query Completion

Get the embeddings for the text.

In [13]:
# models
EMBEDDING_MODEL = "text-embedding-ada-002"
GPT_MODEL = "gpt-3.5-turbo"

# Get OpenAI key
open_ai_key = pd.read_csv('D:\OneDrive\Security\keys.csv')
open.ai_key = open_ai_key[open_ai_key['Organization'] == 'Open_AI']['Key'][0]

client = OpenAI(api_key = open.ai_key)
client

In [14]:
def get_embedding(text_to_embed):
    """Get the embeddings for the text"""

    response = client.embeddings.create(
        model="text-embedding-ada-002",
        input=text_to_embed
    )

    return response

In [15]:
# Call the embeddings function
response = get_embedding(df['text'].tolist())
print(len(response.data))

104


In [16]:
# Add the embeddings to the dataframe
df['embeddings'] = response.data
df

,text,embeddings
0,2 January – The Japan Meteorological Agency (J...,"Embedding(embedding=[-0.0031084921211004257, -..."
1,3 January – The first functional semiconductor...,"Embedding(embedding=[0.002213209867477417, 0.0..."
2,4 January – A review indicates digital rectal ...,"Embedding(embedding=[-0.023844994604587555, 0...."
3,5 January Scientists report that newborn galax...,"Embedding(embedding=[0.004182404838502407, 0.0..."
4,5 January An analysis of sugar-sweetened bever...,"Embedding(embedding=[0.005126591306179762, -0...."
...,...,...
99,8 May Atmospheric gases surrounding 55 Cancri ...,"Embedding(embedding=[0.010923474095761776, 0.0..."
100,9 May – A record annual increase in atmospheri...,"Embedding(embedding=[-0.006725318729877472, -0..."
101,10 May – A series of solar storms and intense ...,"Embedding(embedding=[-0.011255239136517048, -0..."
102,"13 May – OpenAI reveals GPT-4o, its latest AI ...","Embedding(embedding=[-0.022428754717111588, -0..."


In [17]:
# Check and make sure they are ok. You expect a length of 1536
temp = df['embeddings'].iloc[0]
len(temp.dict()['embedding'])

1536

In [18]:
# Did a dir on the object. There is a dict method. Use that to get the embeddings.
print(dir(df['embeddings'].iloc[0]))

['__abstractmethods__', '__annotations__', '__class__', '__class_getitem__', '__class_vars__', '__copy__', '__deepcopy__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__fields__', '__fields_set__', '__format__', '__ge__', '__get_pydantic_core_schema__', '__get_pydantic_json_schema__', '__getattr__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__pretty__', '__private_attributes__', '__pydantic_complete__', '__pydantic_core_schema__', '__pydantic_custom_init__', '__pydantic_decorators__', '__pydantic_extra__', '__pydantic_fields_set__', '__pydantic_generic_metadata__', '__pydantic_init_subclass__', '__pydantic_parent_namespace__', '__pydantic_post_init__', '__pydantic_private__', '__pydantic_root_model__', '__pydantic_serializer__', '__pydantic_validator__', '__reduce__', '__reduce_ex__', '__repr__', '__repr_args__', '__repr_name__', '__repr_str__', '__rich_r

In [19]:
# Get the embeddings from the object using the dict method
df['embeddings_floats'] = df['embeddings'].apply(lambda x: x.dict()['embedding'])
df

,text,embeddings,embeddings_floats
0,2 January – The Japan Meteorological Agency (J...,"Embedding(embedding=[-0.0031084921211004257, -...","[-0.0031084921211004257, -0.0093239089474082, ..."
1,3 January – The first functional semiconductor...,"Embedding(embedding=[0.002213209867477417, 0.0...","[0.002213209867477417, 0.012299963273108006, -..."
2,4 January – A review indicates digital rectal ...,"Embedding(embedding=[-0.023844994604587555, 0....","[-0.023844994604587555, 0.014986811205744743, ..."
3,5 January Scientists report that newborn galax...,"Embedding(embedding=[0.004182404838502407, 0.0...","[0.004182404838502407, 0.0026388594415038824, ..."
4,5 January An analysis of sugar-sweetened bever...,"Embedding(embedding=[0.005126591306179762, -0....","[0.005126591306179762, -0.003509768983349204, ..."
...,...,...,...
99,8 May Atmospheric gases surrounding 55 Cancri ...,"Embedding(embedding=[0.010923474095761776, 0.0...","[0.010923474095761776, 0.005340510979294777, -..."
100,9 May – A record annual increase in atmospheri...,"Embedding(embedding=[-0.006725318729877472, -0...","[-0.006725318729877472, -0.0025702358689159155..."
101,10 May – A series of solar storms and intense ...,"Embedding(embedding=[-0.011255239136517048, -0...","[-0.011255239136517048, -0.009052279405295849,..."
102,"13 May – OpenAI reveals GPT-4o, its latest AI ...","Embedding(embedding=[-0.022428754717111588, -0...","[-0.022428754717111588, -0.01023862510919571, ..."


In [20]:
# Check and make sure that worked. Should get 1536
len(df['embeddings_floats'].iloc[0])

1536

In [21]:
# Sample question
question = 'Where did life probably begin?'

In [22]:
def get_embeddings(question, df):
    
    # Get embedding for question from openai
    
    question_embeddings = client.embeddings.create(
        input = question, 
        model = "text-embedding-ada-002").data[0].embedding
    
    return question_embeddings

In [23]:
def calc_cosign_similarity(question_embeddings, df):

    # Calculate the cosign similarity between the question and the embeddings

    # Convert 'embeddings_floats' column to numpy array
    embeddings_array = np.array(df['embeddings_floats'].tolist())

    # Calculate cosine similarity
    cosine_sims = [cosine(question_embeddings, embedding) for embedding in embeddings_array]

    # Assign the cosine similarity to a new column in the DataFrame
    df['cosine_sims'] = cosine_sims

    # Sort the DataFrame by 'cosine_sims' column in ascending order
    df.sort_values('cosine_sims', inplace=True)

    # Reset the index of the DataFrame
    df.reset_index(inplace=True, drop=True)

    return df['text'].iloc[0]

In [24]:
# Example usage
question_embeddings = get_embeddings(question, df)
answer = calc_cosign_similarity(question_embeddings, df)
print(answer)

9 January Scientists report studies which seem to support the hypothesis that life may have begun in a shallow lake rather than otherwise - perhaps somewhat like a "warm little pond" originally proposed by Charles Darwin.


## Performance 
The purpose of this is to show the efficacy of RAG. I am going to pull out items from the pandas dataframe. Submit those to ChatGPT and have it return a question. Then I am going to use that question to see what the results are for the 3 different versions. 

- ChatGPT unassisted
- Vector look up
- ChatGPT assisted with RAG

I could use input() and type the questions in but ... I hate typing and this is a far more interesting use of the technology. I could also, change the model that is generating the questions but ... that is for another day. 

In [25]:
def chatgpt_answer(question):

    # https://platform.openai.com/docs/guides/text-generation/chat-completions-api

    # send a ChatCompletion request
    response = client.chat.completions.create(
        model = 'gpt-3.5-turbo',
        messages = [
            {'role': 'user', 
             'content': question}
        ],
        temperature = 0
    )
    
    return response

In [26]:
def question_answer_comparison(question, df):   
    
    # Using vector look up
    question_embeddings = get_embeddings(question, df)
    answer = calc_cosign_similarity(question_embeddings, df)
    print('\n', 'Using cosine similarity\n', answer)
    
    # GPT3 vanilla
    response = chatgpt_answer(question)
    print('\nChatGPT3.5 Unassisted\n', response.choices[0].message.content)
    
    # Using rag
    question_embeddings = get_embeddings(question, df)
    response = chatgpt_answer(rag_prompt(question, df))
    print('\nChatGPT3.5 with RAG\n', response.choices[0].message.content)

In [27]:
def rag_prompt(question, df):
    """Ceate a prompt that has the content from the pandas df"""
    
    # Get the embeddings
    question_embeddings = get_embeddings(question, df)
    
    # Get the text
    text = calc_cosign_similarity(question_embeddings, df)
    
    # Create the prompt
    message = f"You are an experienced scientific researcher. {question} Please consider this \
information when formulating your response. Pay particular attention to the date. \
I am looking for the most recent information. The dates in this text are all 2024. {text}. \
Restrict your answer to ONLY what is in this text."
    
    # Get rid of extra spaces
    message = message.strip(' ')
    
    return message

In [28]:
def summarize_prompt(df):
    
    """
    input df - pandas dataframe
    
    Does the following:
    Takes in the df. 
    Randomly selects a text item. 
    Creates a prompt from the text
    Submits it to ChatGPT and asks it to summarize and turn it into a question.
    
    output question - str
    """
    
    # Get a random piece of txt
    text = df.sample(1)['text'].iloc[0]
    
    # Create prompt
    message = f"You are an experienced scientific researcher. Formulate a Question based on this {text}. \
In the Question use at least some of these words 'who, when, where, how much, what quantity'."
    
    # Submit to ChatGPT
    response = chatgpt_answer(message)
    
    # Just the response
    question = response.choices[0].message.content
    
    # Just get the question
    question = question.split('\n\n')[-1]
    
    return question

### Questions

In [29]:
# Lets ask some questions
questions_list = [summarize_prompt(df) for _ in range(5)]
questions_list

['Who are the individuals with low incomes living in small-scale societies outside mainstream societies who report high life satisfaction, and how much of a discrepancy exists between their reported life satisfaction and the conclusions of the 2023 adversarial collaboration study?',
 'How much did the inability of Gigantopithecus blacki to adapt to a changed environment contribute to its extinction, and what quantity of evidence supports this conclusion?',
 'How much mass does the heavier object in the potential radio pulsar-black hole binary system located in the globular cluster NGC 1851 have, and how does it fit within the "mass gap" between neutron stars and black holes?',
 'How much water did Venus lose due to HCO+ dissociative recombination, and when and where did this process occur?',
 'How much older is Tyrannosaurus mcraeensis compared to Tyrannosaurus rex, and where and when did this species exist in relation to the evolution of the Tyrannosaurus genus?']

In [30]:
for idx, question in enumerate(questions_list):
    print(idx + 1, '\n', question)
    print(question_answer_comparison(question, df), '\n#########\n')

1 
 Who are the individuals with low incomes living in small-scale societies outside mainstream societies who report high life satisfaction, and how much of a discrepancy exists between their reported life satisfaction and the conclusions of the 2023 adversarial collaboration study?

 Using cosine similarity
 5 February A study reports high life satisfaction in people with low incomes among small-scale societies outside mainstream societies, in contrast with conclusions of a 2023 adversarial collaboration.

ChatGPT3.5 Unassisted
 The individuals with low incomes living in small-scale societies outside mainstream societies who report high life satisfaction are typically those who have strong social connections, a sense of community, and a deep connection to their cultural traditions and values. These individuals often prioritize relationships, community support, and a sense of purpose over material wealth and possessions.

In the 2023 adversarial collaboration study, researchers may hav


ChatGPT3.5 with RAG
 Tyrannosaurus mcraeensis is reported to have lived 5-7 million years before Tyrannosaurus rex. This means that Tyrannosaurus mcraeensis is older than Tyrannosaurus rex. The discovery of Tyrannosaurus mcraeensis may be fundamentally important to the evolution of the Tyrannosaurus genus.
None 
#########



## Conclusions
ChatGPT unassisted is amazingly good at lying. When it does not know the answer, it simply dreams something up. Since, ChatGPT is  so good at lying and it is trained on a large corpus, some of its "lies" may actually be truths. Just because it happened sometime on or before January 2022 does not mean it is not true today. It still could be true and part of a valid response. 

When I asked ChatGPT with RAG to restrict its answers to the input text it did not lie anymore and ws very accurate. I ran this code several times and on average ChatGPT 3.5 Unassisted got ~ 30% wrong. That is almost certainly a problem for most Chatbot applications. ChatGPT 3.5 With RAG was always correct but ... could not answer the question completely sometimes because the question posed by ChatGPT 3.5 Unassisted wanted information that was not in the RAG prompt. This was not common but did happen. 

When giving the LLM context, I did not return more than one result with this implementation. It is common to return the top several results to give the Large Language Model (LLM) more to work with. Since the sample size was so small and the individual pieces of information on the page were not expected to be closesly related, this is fine. However, depending on the application this may not be and would probably require some more prompt engineering to make sure that the application performed consistently well. 